<a href="https://colab.research.google.com/github/agungtriu/Belajar-Machine-Learning-untuk-Pemula/blob/master/Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2020-07-07 16:16:34--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  5.86MB/s    in 48s     

2020-07-07 16:17:23 (6.47 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [23]:
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors'

file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [24]:
import pandas as pd

In [104]:
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

tag
paper                1424
rock                 1452
rockpaperscissors       1
rps-cv-images           1
scissors             1500
dtype: int64

In [105]:
df = df.set_index("tag")
df = df.drop("rockpaperscissors", axis=0)
df = df.drop("rps-cv-images", axis=0)
df = df.reset_index()
df.groupby(['tag']).size()

tag
paper       1424
rock        1452
scissors    1500
dtype: int64

In [106]:
df.head()

,tag,path,file_name
0,rock,/tmp/rockpaperscissors/rock/DJUhF4lvtGZoaatl.png,DJUhF4lvtGZoaatl.png
1,rock,/tmp/rockpaperscissors/rock/GRUlZRON6TdcmAOe.png,GRUlZRON6TdcmAOe.png
2,rock,/tmp/rockpaperscissors/rock/d9UIt2WF1O9DYHPA.png,d9UIt2WF1O9DYHPA.png
3,rock,/tmp/rockpaperscissors/rock/lZcZsaDTx2vWJFun.png,lZcZsaDTx2vWJFun.png
4,rock,/tmp/rockpaperscissors/rock/T6biQxOCDlM5ad7k.png,T6biQxOCDlM5ad7k.png


In [107]:
from sklearn.model_selection import train_test_split

In [108]:
X = df['path']
y = df['tag']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=300)

In [110]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=100)

In [111]:
df_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'train'})
df_te = pd.DataFrame({'path':X_test, 'tag':y_test, 'set':'test'})
df_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

In [112]:
df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)\

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

#cek sample datanya
df_all.sample(3)


set         tag     
test        paper        147
            rock         133
            scissors     158
train       paper       1124
            rock        1168
            scissors    1208
validation  paper        153
            rock         151
            scissors     134
dtype: int64 




,path,tag,set
841,/tmp/rockpaperscissors/rps-cv-images/rock/weNe...,rock,train
487,/tmp/rockpaperscissors/rps-cv-images/scissors/...,scissors,train
4198,/tmp/rockpaperscissors/rps-cv-images/paper/B9P...,paper,validation


In [113]:
import shutil
from tqdm.notebook import tqdm as tq

In [114]:
datasource_path = "/tmp/rockpaperscissors/"
dataset_path = "/tmp/dataset_rockpaperscissors/"

In [115]:
for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [116]:
base_dir = '/tmp/dataset_rockpaperscissors'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [117]:
os.listdir('/tmp/dataset_rockpaperscissors/train')

['rock', 'scissors', 'paper']

In [118]:
os.listdir('/tmp/dataset_rockpaperscissors/validation')

['rock', 'scissors', 'paper']

In [119]:
train_rock_dir = os.path.join(train_dir, 'rock')
train_scissors_dir = os.path.join(train_dir, 'scissors')
train_paper_dir = os.path.join(train_dir, 'paper')


validation_rock_dir = os.path.join(validation_dir, 'rock')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')
validation_paper_dir = os.path.join(validation_dir, 'paper')

In [120]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [121]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

Found 2101 images belonging to 3 classes.
Found 414 images belonging to 3 classes.


In [122]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [123]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [124]:
model.fit(
      train_generator,
      steps_per_epoch=25,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=5,
      verbose=2)

Epoch 1/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 2/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 3/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 4/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 5/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 6/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 7/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 8/20
25/25 - 6s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 9/20
25/25 - 7s - loss: 1.1921e-07 - accuracy: 0.6667 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 10/20
25/25 - 6s - loss: 1.1921e-07 - ac

In [126]:
test_dir = os.path.join(base_dir, 'test')

In [127]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')

Found 417 images belonging to 3 classes.


In [130]:
scores = model.evaluate(test_generator, verbose=1)

105/105 [==============================] - 8s 80ms/step - loss: 1.1921e-07 - accuracy: 0.6667
